In [1]:
!pip install pyspark


In [2]:
import pyspark
#py4javaerror need this
import findspark
findspark.init()


In [3]:
from pyspark import SparkContext
#create spark context
sc=SparkContext.getOrCreate()

# driver is closing forcebly so I had to run for small set
hospitals_rdd=sc.textFile("test.csv",32)
#display data from RDD
hospitals_rdd.collect()

['hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,is_metro_micro',
 '10055,5/8/2020,AL,10055,FLOWERS HOSPITAL,4370 WEST MAIN STREET,DOTHAN,36305,Short Term,1069,TRUE',
 '21301,6/5/2020,AK,21301,PROVIDENCE VALDEZ MEDICAL CENTER,PO BOX 550,VALDEZ,99686,Critical Access Hospitals,2260,FALSE',
 '50014,4/24/2020,CA,50014,SUTTER AMADOR HOSPITAL,200 MISSION BLVD,JACKSON,95642,Short Term,6005,FALSE',
 '62014,6/19/2020,CO,62014,VIBRA HOSPITAL OF DENVER,8451 PEARL ST,THORNTON,80229,Long Term,8001,TRUE',
 '150034,5/15/2020,FL,100087,SARASOTA MEMORIAL HOSPITAL,1700 S TAMIAMI TRL,SARASOTA,34239,Short Term,12115,TRUE',
 '100118,5/8/2020,FL,100118,ADVENTHEALTH PALM COAST,60 MEMORIAL MEDICAL PKWY,PALM COAST,32164,Short Term,12035,TRUE',
 '100125,6/19/2020,FL,100125,HOMESTEAD HOSPITAL,975 BAPTIST WAY,HOMESTEAD,33033,Short Term,12086,TRUE',
 '100137,4/24/2020,FL,100137,ADVENTHEALTH HEART OF FLORIDA,40100 HWY 27,DAVENPORT,33837,Short Term,12105,TRUE',
 '1002

In [4]:
#Store the header row in a broadcast variable 
header=sc.broadcast(hospitals_rdd.first())
print(header.value)

hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,is_metro_micro


In [5]:
#get/filter the content information without header
hospitals_rdd = hospitals_rdd.filter(lambda x: x != header.value)
hospitals_rdd.collect()

['10055,5/8/2020,AL,10055,FLOWERS HOSPITAL,4370 WEST MAIN STREET,DOTHAN,36305,Short Term,1069,TRUE',
 '21301,6/5/2020,AK,21301,PROVIDENCE VALDEZ MEDICAL CENTER,PO BOX 550,VALDEZ,99686,Critical Access Hospitals,2260,FALSE',
 '50014,4/24/2020,CA,50014,SUTTER AMADOR HOSPITAL,200 MISSION BLVD,JACKSON,95642,Short Term,6005,FALSE',
 '62014,6/19/2020,CO,62014,VIBRA HOSPITAL OF DENVER,8451 PEARL ST,THORNTON,80229,Long Term,8001,TRUE',
 '150034,5/15/2020,FL,100087,SARASOTA MEMORIAL HOSPITAL,1700 S TAMIAMI TRL,SARASOTA,34239,Short Term,12115,TRUE',
 '100118,5/8/2020,FL,100118,ADVENTHEALTH PALM COAST,60 MEMORIAL MEDICAL PKWY,PALM COAST,32164,Short Term,12035,TRUE',
 '100125,6/19/2020,FL,100125,HOMESTEAD HOSPITAL,975 BAPTIST WAY,HOMESTEAD,33033,Short Term,12086,TRUE',
 '100137,4/24/2020,FL,100137,ADVENTHEALTH HEART OF FLORIDA,40100 HWY 27,DAVENPORT,33837,Short Term,12105,TRUE',
 '100260,5/15/2020,FL,100260,HCA FLORIDA ST LUCIE HOSPITAL,1800 SE TIFFANY AVE,PORT SAINT LUCIE,34952,Short Term,12111,TR

In [6]:
#extract rows in form of key and set of rows
def extract_hospital_data(y, header):
    #split values string by comma
    data_list = y.split(",")
    #extract the key value
    key = data_list[0]
    #get remaining values except key 
    values = data_list[1:]
    #split the header string by comma
    header_list = header.split(",")
    
    #create a set with header column and value except key into dictionary
    values_dict = dict(zip(header_list[1:], values))
    return (key, values_dict)

hospitals_rdd = hospitals_rdd.map(lambda y: extract_hospital_data(y, header.value))
result = hospitals_rdd.collect()
print(result)

[('10055', {'collection_week': '5/8/2020', 'state': 'AL', 'ccn': '10055', 'hospital_name': 'FLOWERS HOSPITAL', 'address': '4370 WEST MAIN STREET', 'city': 'DOTHAN', 'zip': '36305', 'hospital_subtype': 'Short Term', 'fips_code': '1069', 'is_metro_micro': 'TRUE'}), ('21301', {'collection_week': '6/5/2020', 'state': 'AK', 'ccn': '21301', 'hospital_name': 'PROVIDENCE VALDEZ MEDICAL CENTER', 'address': 'PO BOX 550', 'city': 'VALDEZ', 'zip': '99686', 'hospital_subtype': 'Critical Access Hospitals', 'fips_code': '2260', 'is_metro_micro': 'FALSE'}), ('50014', {'collection_week': '4/24/2020', 'state': 'CA', 'ccn': '50014', 'hospital_name': 'SUTTER AMADOR HOSPITAL', 'address': '200 MISSION BLVD', 'city': 'JACKSON', 'zip': '95642', 'hospital_subtype': 'Short Term', 'fips_code': '6005', 'is_metro_micro': 'FALSE'}), ('62014', {'collection_week': '6/19/2020', 'state': 'CO', 'ccn': '62014', 'hospital_name': 'VIBRA HOSPITAL OF DENVER', 'address': '8451 PEARL ST', 'city': 'THORNTON', 'zip': '80229', 'h

In [14]:
# reduceByKey to group rows by hospital pk
def group_rows(r1, r2):
    return [r1[0], r1[1].union(r2[1])]

grouped_rdd = hospitals_rdd.reduceByKey(group_rows)
print(grouped_rdd.collect())

[('10055', {'collection_week': '5/8/2020', 'state': 'AL', 'ccn': '10055', 'hospital_name': 'FLOWERS HOSPITAL', 'address': '4370 WEST MAIN STREET', 'city': 'DOTHAN', 'zip': '36305', 'hospital_subtype': 'Short Term', 'fips_code': '1069', 'is_metro_micro': 'TRUE'}), ('111302', {'collection_week': '4/24/2020', 'state': 'GA', 'ccn': '111302', 'hospital_name': 'BLECKLEY MEMORIAL HOSPITAL', 'address': '145 PEACOCK STREET', 'city': 'COCHRAN', 'zip': '31014', 'hospital_subtype': 'Critical Access Hospitals', 'fips_code': '13023', 'is_metro_micro': 'FALSE'}), ('241326', {'collection_week': '7/3/2020', 'state': 'ID', 'ccn': '130013', 'hospital_name': 'SAINT ALPHONSUS MEDICAL CENTER - NAMPA', 'address': '4300 E FLAMINGO AVE', 'city': 'NAMPA', 'zip': '83687', 'hospital_subtype': 'Short Term', 'fips_code': '16027', 'is_metro_micro': 'TRUE'}), ('111305', {'collection_week': '5/29/2020', 'state': 'GA', 'ccn': '111305', 'hospital_name': 'MILLER COUNTY HOSPITAL', 'address': '209 N CUTHBERT STREET', 'city

In [19]:

# Define a list of hospital pks from the question
hospital_pks = ['150034', '050739', '330231','241326', '070008']

# Iterate over each hospital pk in the list  
keyvaluepair_data = []    
for key, value in grouped_rdd.take(grouped_rdd.count()):
    #print(value)
    if(key in hospital_pks):
        keyvaluepair_data.append((key, value))
print(keyvaluepair_data)
    

[('241326', {'collection_week': '7/3/2020', 'state': 'ID', 'ccn': '130013', 'hospital_name': 'SAINT ALPHONSUS MEDICAL CENTER - NAMPA', 'address': '4300 E FLAMINGO AVE', 'city': 'NAMPA', 'zip': '83687', 'hospital_subtype': 'Short Term', 'fips_code': '16027', 'is_metro_micro': 'TRUE'}), ('150034', {'collection_week': '5/15/2020', 'state': 'FL', 'ccn': '100087', 'hospital_name': 'SARASOTA MEMORIAL HOSPITAL', 'address': '1700 S TAMIAMI TRL', 'city': 'SARASOTA', 'zip': '34239', 'hospital_subtype': 'Short Term', 'fips_code': '12115', 'is_metro_micro': 'TRUE'}), ('330231', {'collection_week': '5/8/2020', 'state': 'GA', 'ccn': '110177', 'hospital_name': 'DOCTORS HOSPITAL', 'address': '3651 WHEELER ROAD', 'city': 'AUGUSTA', 'zip': '30909', 'hospital_subtype': 'Short Term', 'fips_code': '13245', 'is_metro_micro': 'TRUE'})]


In [ ]:
#TASK 2

In [52]:
#source reference: http://ekzhu.com/datasketch/minhash.html

from datasketch import MinHash

hashes={}
#loop through all the hospital records, update minhash with the value
#log the minhash for the key and display the matched key values
for key, value in grouped_rdd.take(grouped_rdd.count()):
  mh=MinHash(num_perm=100)
  for d in value:
    mh.update(d.encode('utf8'))
  hashes[key]=mh

for key, value in hashes.items():
  keys=['150034', '050739', '330231','241326', '070008']
  if key in keys:
    print(f'{key}:{value}')

241326:<datasketch.minhash.MinHash object at 0x000001BF983A6CA0>
150034:<datasketch.minhash.MinHash object at 0x000001BF983670D0>
330231:<datasketch.minhash.MinHash object at 0x000001BF98367760>


In [ ]:
#TASK 3

In [54]:
#source reference: http://ekzhu.com/datasketch/lsh.html
from datasketch import MinHash, MinHashLSH

jaccard={}
# Create LSH index

keys=['150034', '330231','241326']
for i in keys:
  lsh = MinHashLSH(threshold=0.2, num_perm=100)
  # loop through hash items and identify keys that do not match
  # put those into the jaccard dictionary
  for key,value in hashes.items():
    if key not in keys:
      lsh.insert(key, value)
    result = lsh.query(hashes[i])
    jaccard[i]=result
for key,value in jaccard.items():
    print(f'{key}  :  {value[:20]}')

150034  :  ['121302', '100118', '110101', '100137', '111305', '10055', '62014', '50014', '110233', '111302', '111312', '111314', '110045', '21301', '100260', '100125']
330231  :  ['121302', '100118', '110101', '100137', '111305', '10055', '62014', '50014', '110233', '111302', '111312', '111314', '110045', '21301', '100260', '100125']
241326  :  ['121302', '100118', '110101', '100137', '111305', '10055', '62014', '50014', '110233', '111302', '111312', '111314', '110045', '21301', '100260', '100125']


In [50]:
hashes.items()

dict_items([('10055', <datasketch.minhash.MinHash object at 0x000001BF983D1520>), ('111302', <datasketch.minhash.MinHash object at 0x000001BF983D1190>), ('241326', <datasketch.minhash.MinHash object at 0x000001BF983FF340>), ('111305', <datasketch.minhash.MinHash object at 0x000001BF98379E80>), ('21301', <datasketch.minhash.MinHash object at 0x000001BF98379310>), ('100118', <datasketch.minhash.MinHash object at 0x000001BF983799A0>), ('50014', <datasketch.minhash.MinHash object at 0x000001BF983797F0>), ('100137', <datasketch.minhash.MinHash object at 0x000001BF983793A0>), ('110045', <datasketch.minhash.MinHash object at 0x000001BF98379F40>), ('150034', <datasketch.minhash.MinHash object at 0x000001BF983792E0>), ('100125', <datasketch.minhash.MinHash object at 0x000001BF98379A30>), ('110101', <datasketch.minhash.MinHash object at 0x000001BF98379EE0>), ('100260', <datasketch.minhash.MinHash object at 0x000001BF98379400>), ('330231', <datasketch.minhash.MinHash object at 0x000001BF98379DC0>